In [1]:
# Team stats instead of player, or some transformation thereof
from collections import defaultdict, Counter
from datetime import datetime
from src.data import load_cache_entry
from src.config import config
from src.analysis import build_owner_lookup

LEAGUE_ID = config.league_id

# Mapping proTeamId to readable name (ESPN codes 0=FA, 1=Atl, 2=Buf, etc.)
PRO_TEAM_MAP = {
    1: "Falcons", 2: "Bills", 3: "Bears", 4: "Bengals", 5: "Browns", 6: "Cowboys",
    7: "Broncos", 8: "Lions", 9: "Packers", 10: "Titans", 11: "Colts", 12: "Chiefs",
    13: "Raiders", 14: "Rams", 15: "Dolphins", 16: "Vikings", 17: "Patriots",
    18: "Saints", 19: "Giants", 20: "Jets", 21: "Eagles", 22: "Cardinals",
    23: "Steelers", 24: "Chargers", 25: "49ers", 26: "Seahawks", 27: "Buccaneers",
    28: "Washington", 29: "Panthers", 30: "Jaguars", 33: "Ravens", 34: "Texans"
}

def fanboy_index(league_id, years, max_periods=20, current_week_cap=None):
    """
    Count starter slots per NFL pro team for each fantasy team (owner).
    """
    current_year = datetime.now().year
    fanboy = defaultdict(Counter)

    for year in years:
        limit = max_periods
        if current_week_cap and year == current_year:
            limit = min(limit, current_week_cap)

        for period in range(1, limit+1):
            data = load_cache_entry(league_id, year, period, ["mMatchup","mTeam"])
            if not data:
                continue

            for row in data.get("schedule", []):
                if row.get("matchupPeriodId") != period:
                    continue
                for side in ["home","away"]:
                    team = row.get(side)
                    if not team:
                        continue
                    tid = team["teamId"]
                    entries = team.get("rosterForMatchupPeriod", {}).get("entries", [])
                    for e in entries:
                        slot = e.get("lineupSlotId")
                        if slot in (20, 21):  # skip bench/IR
                            continue
                        player = e.get("playerPoolEntry", {}).get("player", {})
                        pro_id = player.get("proTeamId")
                        if pro_id is None:
                            continue
                        pro_name = PRO_TEAM_MAP.get(pro_id, f"Team{pro_id}")
                        fanboy[tid][pro_name] += 1

    return fanboy

# ---- Unit Test ----
years_to_check = list(range(2018,2026))
# Build owner lookup from most recent season in cache
latest_year = max(years_to_check)
latest_data = load_cache_entry(LEAGUE_ID, latest_year, 1, ["mMatchup","mTeam"])
owner_lookup = build_owner_lookup(latest_data)

# Compute fanboy index
fanboys = fanboy_index(
    LEAGUE_ID,
    years_to_check,
    max_periods=20,
    current_week_cap=2,
)

# Display top 5 per owner
for tid, counts in fanboys.items():
    owner = owner_lookup.get(tid, f"Team {tid}")
    print(f"\n=== {owner} fanboy index ===")
    for team, cnt in counts.most_common(5):
        print(f"  {team}: {cnt}")



=== Froneberger fanboy index ===
  Panthers: 76
  Seahawks: 60
  Cardinals: 59
  Washington: 55
  Bengals: 55

=== Trandel fanboy index ===
  49ers: 70
  Cowboys: 66
  Raiders: 54
  Rams: 52
  Dolphins: 49

=== McDaniel fanboy index ===
  Chargers: 76
  Saints: 68
  Buccaneers: 53
  Texans: 48
  Raiders: 48

=== Tanner fanboy index ===
  Packers: 136
  Steelers: 86
  Colts: 57
  Buccaneers: 50
  Ravens: 49

=== Randall fanboy index ===
  Buccaneers: 76
  Seahawks: 65
  Falcons: 55
  Ravens: 50
  Saints: 50

=== Grogan fanboy index ===
  Lions: 94
  Chiefs: 91
  Saints: 65
  Rams: 65
  Chargers: 59

=== Roberts fanboy index ===
  Ravens: 95
  Bengals: 85
  Dolphins: 80
  Packers: 57
  Texans: 54

=== Gantt fanboy index ===
  Patriots: 87
  Chiefs: 79
  Bills: 79
  Falcons: 57
  Ravens: 53

=== LeFeve fanboy index ===
  Bills: 94
  Eagles: 87
  Cardinals: 87
  Ravens: 60
  Rams: 50

=== Matz fanboy index ===
  Falcons: 54
  Bears: 54
  Washington: 50
  Vikings: 48
  Steelers: 47

=== Fe

In [2]:
import pandas as pd

# Convert fanboy Counter dict → DataFrame
def fanboy_df_from_counters(fanboys, owner_lookup):
    """
    Transform {teamId: Counter(pro_team: starts)} into a DataFrame
    Rows = owners, Cols = NFL teams, Values = counts
    """
    records = {}
    for tid, counter in fanboys.items():
        owner = owner_lookup.get(tid, f"Team {tid}")
        records[owner] = dict(counter)
    df = pd.DataFrame.from_dict(records, orient="index").fillna(0).astype(int)
    # Order columns alphabetically for consistency
    df = df.reindex(sorted(df.columns), axis=1)
    return df

# ---- Build the DataFrame ----
fanboy_df = fanboy_df_from_counters(fanboys, owner_lookup)

print("📊 Fanboy DataFrame shape:", fanboy_df.shape)
display(fanboy_df.head())  # preview first few rows


📊 Fanboy DataFrame shape: (12, 33)


,49ers,Bears,Bengals,Bills,Broncos,Browns,Buccaneers,Cardinals,Chargers,Chiefs,...,Rams,Ravens,Saints,Seahawks,Steelers,Team0,Texans,Titans,Vikings,Washington
Froneberger,53,6,55,9,52,10,34,59,51,35,...,32,34,16,60,19,1,31,35,31,55
Trandel,70,17,17,37,20,21,48,16,39,39,...,52,26,40,38,31,1,36,24,37,20
McDaniel,39,37,18,21,26,28,53,36,76,45,...,14,28,68,38,22,5,48,40,44,12
Tanner,18,24,29,31,45,15,50,32,7,1,...,41,49,18,13,86,10,40,24,39,42
Randall,25,38,12,17,17,25,76,33,33,45,...,36,50,50,65,48,11,28,42,30,27


In [ ]:
from src import viz

viz.plot_fanboy_heatmap(fanboy_df, "fanboy_charts/fanboy_heatmap.png")
viz.plot_fanboy_stacked_bars(fanboy_df, "fanboy_charts/fanboy_stacked.png", top_n=5)
# viz.plot_fanboy_pies(fanboy_df, "charts", top_n=8) # Disabling, not good



In [ ]:
# League-wide bar trophy
viz.plot_fanboy_leaguewide(fanboy_df, "fanboy_charts/fanboy_trophy.png", top_n=32)

# Sankey (owners → NFL teams)
viz.plot_fanboy_sankey(fanboy_df, "fanboy_charts/fanboy_sankey.png", top_n=5)

/home/alex/Documents/FFL/ESPN-FFL/src/viz.py:137: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




⚠️ Could not save PNG, wrote interactive HTML instead: charts/fanboy_sankey.html
